## VQE apply to Quantum Chemistry: Ground Eigenstate of $H_{3}$

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

### We define the molecule of we are going to calculate the gorundstate, this is $H_{3}$

In [5]:
symbols = ["H", "H", "H"]

# We specify where th enuclei are located
# Database for molecular cooridantes: https://cccbdb.nist.gov/alldata1x.asp
cooridnates = np.array([[0.0102, 0.0442, 0.0],
                        [0.9867, 1.6303, 0.0],
                        [1.8720, -0.0085, 0.0]])

### Finding the molecular Hamiltonian

In [6]:
# we define the molecular hamiltonian

# In VQE is easier to work with two electrons than other number or electrons or a neutral molecule.
# For it we set charge = 1
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols,
                                                  cooridnates,
                                                  charge = 1)

In [7]:
# Let's see how many qubits we need to simulate this molecule

print(qubits)

6


In [8]:
# We define a non exactly ground state known as Hartree-Fock State,
# this will be an initial aproximation to ground state.

hf = qchem.hf_state(electrons = 2, orbitals = 6)

# This result are going to be printing in Jordan Wegner representation
print(hf)

[1 1 0 0 0 0]


In [9]:
num_wires = qubits
dev = qml.device("default.qubit", wires = num_wires)

In [10]:
# The circuit we are going to use to encode and construct the ansatz

@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state), wires = range(num_wires))
    return qml.expval(hamiltonian)

In [11]:
exp_energy(hf)

tensor(-1.24655016, requires_grad=True)

### Preparing the trial ground state

In [12]:
def ansatz(params):
    qml.BasisState(hf, wires = range(num_wires))
    qml.DoubleExcitation(params[0], wires = [0,1,2,3])
    qml.DoubleExcitation(params[1], wires = [0,1,4,5])

### Minimazing the expectation value of the Hamiltonian

We use the \italics{Ritz variational principle}, which leds:

$$E_{0} \leq min \langle \phi | \hat{H} | \phi \rangle$$

In [13]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)

In [14]:
cost_function([0.1, 0.1])

tensor(-1.26796721, requires_grad=True)

In [15]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array([0.0,0.0], requires_grad = True)

energy = [cost_function(theta)]
angle = [theta]
max_iterations = 20

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)
    
    energy.append(cost_function(theta))
    angle.append(theta)
    
    if n%2 == 0:
        print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")

Step = 0, Energy = -1.26070025 Ha
Step = 2, Energy = -1.27115671 Ha
Step = 4, Energy = -1.27365804 Ha
Step = 6, Energy = -1.27425241 Ha
Step = 8, Energy = -1.27439362 Ha
Step = 10, Energy = -1.27442718 Ha
Step = 12, Energy = -1.27443517 Ha
Step = 14, Energy = -1.27443707 Ha
Step = 16, Energy = -1.27443752 Ha
Step = 18, Energy = -1.27443763 Ha


In [16]:
print("\n" f"Final ground energy: {energy[-1]:.8f} Ha")
print("\n" f"Final angle parameters: {theta[0]:.8f} {theta[1]:.8f}")


Final ground energy: -1.27443764 Ha

Final angle parameters: 0.19203468 0.19290335


In [17]:
# What the actual groundstate is? 

@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

In [18]:
ground_state(theta)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09585862+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09586987+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99076743+0.j,  0.        +0.j,  0.        +0.

that state si the following if we look closer:

$$| ground state \rangle = 0.9908|110000\rangle - 0.096|001100\rangle - 0.096|000011\rangle$$